# Generate Lists for Neural Net

In [3]:
import csv
import bq
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb

In [4]:
client = bq.Client.Get()

def Query(q):
    t0 = time.time()
    answer = client.ReadTableRows(client.Query(q)['configuration']['query']['destinationTable'])
    print 'Query time: ' + str(time.time() - t0) + ' seconds.'
    return answer

## Get Vessels that Have Been Matched to the CLAV List

In [5]:
q = '''select shiptype, count(*) num from [world-fishing-827:CLAV_match_results.v7_results] group by shiptype
       order by num desc '''
geartypes = Query(q)

Waiting on bqjob_r486963b3657b6076_00000157d0a8ebb9_1 ... (0s) Current status: DONE   
Query time: 1.8374350071 seconds.


In [6]:
for g in geartypes:
    print g[0],g[1]

Longliners 2057
Tuna longliners 975
Purse seiners 412
Fish carriers 406
Trawlers 392
Tuna purse seiners 282
Pole and Line vessels 203
Non-fishing vessels nei 146
Unknown 127
Trollers 86
Recreational fishing vessels 52
Multipurpose vessels 47
Support Vessels 42
Bunkers 41
Fishery research vessels 37
Gillnetters 30
Multipurpose vessels nei 28
Dredgers 8
None 5
Handliners 4
Liners nei 3
Liners 3
Motherships 2
Seiners 2
Stern trawlers 1
Seiners nei 1
Other line vessels 1


## Rules for CLAV Shiptypes

In [7]:
clav_rules = {
'Longliners': 'Longliners',
'Tuna longliners': 'Longliners',
'Purse seiners': 'Purse seiners',
'Trawlers': 'Trawlers',
'Trawler':'Trawlers',
'Trollers':'Trollers' ,
'Pole and Line vessels': 'Pole and Line',
'Tuna purse seiners':'Purse seiners',
'Gillnetters':'Set gillnets'}
    
# ignore the following cageories
'''
Fish carriers
Non-fishing vessels nei 146
Unknown 127
Recreational fishing vessels 52
Multipurpose vessels 47
Support Vessels 42
Bunkers 41
Fishery research vessels 37
Multipurpose vessels nei 28
Dredgers 8
None 5
Handliners 4
Liners nei 3
Liners 3
Motherships 2
Seiners 2
Stern trawlers 1
Seiners nei 1
Other line vessels'''
None

In [8]:
# get the importand data from the CLAV lists
q = '''
select mmsi, group_concat(shiptype) geartype, avg(length),avg(tonnage), 
avg(stddev_length), avg(stddev_tonnage), count(*) number_shiptypes from
(select a.mmsi as mmsi, a.shiptype as shiptype, avg(b.length) length, avg(c.tonnage) tonnage, 
stddev(b.length) stddev_length, stddev(c.tonnage) stddev_tonnage
from 
(SELECT mmsi, shipname, 
CASE
  WHEN shiptype = 'Tuna purse seiners' THEN 'Purse seines'
  WHEN shiptype = 'Purse seiners' THEN 'Purse seines'
  WHEN shiptype = 'Tuna longliners' THEN 'Longliners'
  WHEN shiptype = 'Trawler' THEN 'Trawlers'
  ELSE shiptype
END shiptype, 
clav_TUVI FROM [world-fishing-827:CLAV_match_results.v7_results]) a
left join
(select length, clav_TUVI from [world-fishing-827:Registry_matching_sources.CLAV_12_14_2015] where length is not null)  b
on a.clav_TUVI = b.clav_TUVI
left join 
(select tonnage, clav_TUVI from [world-fishing-827:Registry_matching_sources.CLAV_12_14_2015] where tonnage is not null)  c
on a.clav_TUVI = c.clav_TUVI
group by mmsi, shiptype)
group by mmsi
having number_shiptypes = 1
'''
clav = Query(q)

Waiting on bqjob_r44e4e1c0eacc5bab_00000157d0a8f2f6_2 ... (0s) Current status: DONE   
Query time: 1.64772295952 seconds.


In [9]:
rows = []
mmsis = []

for c in clav:
    if c[1] in clav_rules: # if it is a shiptype we are including
        mmsi = c[0]
        geartype = clav_rules[c[1]] # replace with the right shipname
        length = c[2]
        tonnage = c[3]
        stddev_length = c[4]
        stddev_tonnage = c[5]
        if length !=None: length=float(c[2])
        if tonnage !=None: tonnage = float(c[3])
        if stddev_length !=None: stddev_length = float(c[4])
        if stddev_tonnage !=None: stddev_tonnage = float(c[5])
        if stddev_length !=None and stddev_length/length > .1:
            length = None # if different entries differ by more than 10 percent, ignore
        if stddev_tonnage !=None and stddev_tonnage/tonnage > 1:
            tonnage = None
        rows.append([geartype,length,tonnage])
        mmsis.append(mmsi)

In [10]:
clav_frame = pd.DataFrame(rows,index=mmsis,columns=['clav_geartype', 'clav_length', 'clav_tonnage'])

In [11]:
clav_frame.ix['224025440']

clav_geartype    Longliners
clav_length              25
clav_tonnage             85
Name: 224025440, dtype: object

## Get Vessels that Have Been Matched to the ITU List

In [12]:
# First identify the geartypes

q = '''select itu_Individual_classification, count(*) num from 
[world-fishing-827:Registry_matching_sources.ITU_Dec_2015_Fishing] 
group by itu_Individual_classification
order by 2 desc'''
itu_gears = Query(q)

Waiting on bqjob_r1fcc267b843d5882_00000157d0a8f98a_3 ... (0s) Current status: DONE   
Query time: 1.18878293037 seconds.


In [13]:
for i in itu_gears:
    print i[0],i[1]

Fishing vessel 10352
Trawler 633
None 399
Lobster ship 261
Motor boat 86
Unspecified 57
Passenger ship 47
Cutter 25
Launch 18
Tunny ship 18
Yacht 11
Barge 9
Research ship /Survey ship 9
Reefer 7
Salvage ship 7
Liner 4
Cable ship 3
Coaster 3
Cargo ship 3
Barge carrier 3
Auxiliary ship 3
Sailing ship 2
Whaler 2
RoRo ship 2
Fishing guard 2
Lugger 2
Bulk carrier 2
Factory ship 2
Training ship 2
Air-cushion vehicle 1
Sloop 1
Cargo and passenger 1
Cruiser 1
Supply vessel 1
Lightship 1
Vehicle carrier 1
Dredger 1
Ship used by divers 1
Pontoon 1


In [14]:
# only the following matter
itu_rules = {
  'Trawler':'Trawlers',
  'Lobster ship': 'Pots and Traps',
  'Fishing vessel':'Fishing vessel'
}

In [15]:
q = ''' SELECT
    mmsi,
    itu_Individual_classification,
    tonnage
  FROM
    [world-fishing-827:Registry_matching_sources.ITU_Dec_2015_Fishing]'''
itu = Query(q)

Waiting on bqjob_r3f4687241f67aff6_00000157d0a8fe3b_4 ... (0s) Current status: DONE   
Query time: 2.3676199913 seconds.


In [16]:
rows = []
mmsis = []

for c in itu:
    if c[1] in itu_rules: # if it is a shiptype we are including
        mmsi = c[0]
        geartype = itu_rules[c[1]] # replace with the right shipname
        try:
            tonnage = float(c[2])
        except:
            tonnage = None       
        rows.append([geartype,tonnage])
        mmsis.append(mmsi)

In [17]:
itu_frame = pd.DataFrame(rows,index=mmsis,columns=['itu_geartype', 'itu_tonnage'])

In [18]:
itu_frame.head()

,itu_geartype,itu_tonnage
204211000,Fishing vessel,NaN
204212000,Trawlers,NaN
204213000,Fishing vessel,NaN
204218000,Fishing vessel,NaN
204228000,Fishing vessel,NaN


## Get Vessels that Have Been Matched to the EU List

In [19]:
# first find the geartypes

q = '''select geartype, count(*) from (select a.mmsi, b.Gear_Main_Code geartype, 
b.Loa length, b.Ton_Gt tonnage from 
(SELECT mmsi,row_number FROM [world-fishing-827:EU_match_results.EU_v2]) a
left join [Registry_matching_sources.EU_registry_311215] b
on a.row_number = b.row_number) group by geartype order by 2 desc'''
eu_gear = Query(q)

Waiting on bqjob_raba6975b4d0f89a_00000157d0a907c2_5 ... (0s) Current status: DONE   
Query time: 1.47289395332 seconds.


In [20]:
# see http://ec.europa.eu/fisheries/fleet/index.cfm?method=Codification.Cod_gear
eu_gear_codes = {"DRB":"Boat dredges",
"DRH":"Hand dredges operating from a boat",
"HMD":"Mechanised dredges including suction dredges",
"GNC":"Encircling gillnets",
"GND":"Drift nets",
"GNS":"Set gillnets (anchored)",
"GTN":"Combined gillnets-trammel nets",
"GTR":"Trammel nets",
"HAR":"Harpoons",
"LLS":"Set longlines",
"LLD":"Drifting longlines",
"LHM":"Handlines and pole-lines (mechanised)",
"LHP":"Handlines and pole-lines (hand operated)",
"LLD":"Drifting longlines",
"LLS":"Set longlines",
"LTL":"Trolling lines",
"LNB":"Boat-operated lift nets",
"LNS":"Shore-operated stationary lift nets",
"NO":"No gear",
"SB":"Beach seines",
"SDN":"Danish seines",
"SPR":"Pair seines",
"SSC":"Scottish seines",
"LA":"Lampara nets",
"PS":"Purse seines",
"FPO":"Pots and Traps",
"OTB":"Bottom otter trawls",
"OTM":"Midwater otter trawls",
"OTT":"Otter twin trawls",
"PTB":"Bottom pair trawls",
"PTM":"Midwater pair trawls",
"TBB":"Beam trawls",
"NK":"Unknown gear"};


for e in eu_gear:
    print e[0],eu_gear_codes[e[0]], e[1]

OTB Bottom otter trawls 3118
PS Purse seines 779
TBB Beam trawls 517
GNS Set gillnets (anchored) 487
OTM Midwater otter trawls 311
LLS Set longlines 241
DRB Boat dredges 229
LLD Drifting longlines 206
FPO Pots and Traps 141
GTR Trammel nets 100
OTT Otter twin trawls 66
HMD Mechanised dredges including suction dredges 59
LHP Handlines and pole-lines (hand operated) 48
PTB Bottom pair trawls 43
SDN Danish seines 35
PTM Midwater pair trawls 29
SSC Scottish seines 23
GND Drift nets 16
DRH Hand dredges operating from a boat 14
LTL Trolling lines 7
GTN Combined gillnets-trammel nets 7
SPR Pair seines 3
GNC Encircling gillnets 3
LNB Boat-operated lift nets 2
SB Beach seines 2
NO No gear 1
LA Lampara nets 1


In [21]:
eu_gear2 = {"DRB":"Trawlers",
"DRH":"Trawlers",
"HMD":"Trawlers",
"GNC":"Set gillnets",
"GND":"Fishing vessel",
"GNS":"Set gillnets",
"GTN":"Set gillnets",
"GTR":"Set gillnets",
"HAR":"Fishing vessel",
"LHM":"Pole and line",
"LHP":"Pole and Line",
"LLD":"Drifting longlines",
"LLS":"Set longlines",
"LTL":"Trollers",
"LNB":"Fishing vessel",
"LNS":"Fishing vessel",
"NO":"Fishing vessel",
"SB":"Fishing vessel",
"SDN":"Fishing vessel",
"SPR":"Fishing vessel",
"SSC":"Fishing vessel",
"LA":"Set gillnets",
"PS":"Purse seines",
"FPO":"Pots and Traps",
"OTB":"Trawlers",
"OTM":"Trawlers",
"OTT":"Trawlers",
"PTB":"Trawlers",
"PTM":"Trawlers",
"TBB":"Trawlers",
"NK":"Unknown gear"}

In [22]:
# get EU values

q = '''
select mmsi, group_concat(geartype) geartype, avg(length), avg(tonnage),
count(*) num_mmsi from
(select a.mmsi mmsi, b.Gear_Main_Code geartype, 
b.Loa length, b.Ton_Gt tonnage from 
(SELECT mmsi,row_number FROM [world-fishing-827:EU_match_results.EU_v2]) a
left join [Registry_matching_sources.EU_registry_311215] b
on a.row_number = b.row_number)
group by mmsi
having num_mmsi = 1

'''
eu = Query(q)

Waiting on bqjob_r3f757aacaa1daca_00000157d0a90dac_6 ... (0s) Current status: DONE   
Query time: 2.15252590179 seconds.


In [23]:
rows = []
mmsis = []

for c in eu:
    if c[1] in eu_gear2: # if it is a shiptype we are including
        mmsi = c[0]
        geartype2 = eu_gear2[c[1]] # replace with the right geartype
        geartype = geartype2
        if "longlines" in geartype2:
            geartype = "Longliners"
        length = c[2]
        tonnage = c[3]
        if length !=None: length=float(c[2])
        if tonnage !=None: tonnage = float(c[3])
        rows.append([geartype, geartype2, length, tonnage])
        mmsis.append(mmsi)

In [24]:
eu_frame = pd.DataFrame(rows,index=mmsis,columns=['eu_geartype','eu_geartype2','eu_length', 'eu_tonnage'])

In [25]:
eu_frame.head()

,eu_geartype,eu_geartype2,eu_length,eu_tonnage
247102210,Trawlers,Trawlers,15.09,37.00
207826760,Trawlers,Trawlers,17.00,20.37
237015000,Trawlers,Trawlers,25.90,103.00
219002506,Trawlers,Trawlers,14.56,36.70
224111190,Trawlers,Trawlers,31.30,254.00


## Get Vessels that Have Been Matched to the CCAMLR List

In [26]:
q = '''SELECT geartype, count(*) num 
FROM [world-fishing-827:Registry_matching_sources.CCAMLR_July_2015_with_mmsi] 
group by geartype order by num desc'''
ccamlr = Query(q)

Waiting on bqjob_r12d4b9b461f2f165_00000157d0a9164e_7 ... (0s) Current status: DONE   
Query time: 2.99377918243 seconds.


In [27]:
for c in ccamlr:
    print c[0],c[1]

Longlines (not specified) 23
Midwater trawls (not specified) 9
Other trawls (not specified) 7
Longlines (not specified); Other trawls (not specified) 1
Longlines (not specified); Pots 1


In [28]:
ccamlr_gear = {
'Longlines (not specified)': 'Longliners',
'Midwater trawls (not specified)':"Trawlers", 
'Other trawls (not specified)':'Trawlers' 
}

# ignore the follwoing
# Longlines (not specified); Other trawls (not specified) 1
# Longlines (not specified); Pots 1

In [29]:
q = '''SELECT
    mmsi, geartype, length, tonnage
from  [world-fishing-827:Registry_matching_sources.CCAMLR_July_2015_with_mmsi]
'''
ccamlr = Query(q)

Waiting on bqjob_r10c2381684b7b151_00000157d0a9220b_8 ... (0s) Current status: DONE   
Query time: 1.306235075 seconds.


In [30]:
rows = []
mmsis = []

for c in ccamlr:
    if c[1] in ccamlr_gear: # if it is a shiptype we are including
        mmsi = c[0]
        geartype = ccamlr_gear[c[1]] # replace with the right geartype
        length = c[2]
        tonnage = c[3]
        if length !=None: length=float(c[2])
        if tonnage !=None: tonnage = float(c[3])
        rows.append([geartype, length,tonnage])
        mmsis.append(mmsi)

In [31]:
ccamlr_frame = pd.DataFrame(rows,index=mmsis,columns=['ccamlr_geartype','ccamlr_length', 'ccamlr_tonnage'])

## Squid Vessels

In [32]:
# from spreadsheet

squids = [412420249,412420405,412420536,412430920,412439583,412421005,412000812,412200146,
          412206009,412350801,412354049,412354055,412354057,412354058,412354091,412354167,
          412354168,412354445,412354446,412371203,412371204,412371205,412402273,412420008,
          412420109,412420214,412420225,412420228,412420229,412420231,412420242,412420243,
          412420244,412420245,412420246,412420248,412420252,412420253,412420254,412420255,
          412420277,412420278,412420312,412420315,412420317,412420341,412420378,412420382,
          412420383,412420384,412420406,412420409,412420418,412420422,412420423,412420427,
          412420428,412420432,412420439,412420452,412420453,412420454,412420455,412420456,
          412420457,412420458,412420464,412420466,412420467,412420471,412420488,412420493,
          412420494,412420502,412420503,412420504,412420531,412420532,412420533,412420538,
          412420609,412420622,412420623,412420624,412420644,412420646,412420647,412420652,
          412420688,412420691,412420697,412420789,412420798,412420799,412420801,412420816,
          412420822,412420823,412420825,412420872,412420873,412420923,412420925,412420926,
          412420927,412420928,412420933,412420937,412420951,412420955,412420956,412420957,
          412420958,412420959,412420961,412420962,412420963,412420964,412420965,412420966,
          412420968,412420969,412420971,412420972,412420973,412420974,412420975,412420979,
          412420981,412420992,412420994,412421002,412421003,412421004,412421016,412421033,
          412421356,412421357,412422693,412422695,412422696,412422697,412422701,412422706,
          412424491,412424547,412424702,412424703,412430930,412439584,412439586,412439587,
          412439741,412439742,412670670,412679810,412697150,412857888,416562000,416563000,
          440995000,441767000,373039000,356514000,370178000,411164730]
geartypes = ["Squid" for s in squids]
mmsis = [str(s) for  s in squids]




In [33]:
squid_frame = pd.DataFrame(geartypes,index=mmsis,columns=['squid_geartype'])

# Reefers

In [34]:
q = '''SELECT
  mmsi,
  "Reefer" geartype,
  AVG(length) length,
  AVG(tonnage) tonnage,
  STDDEV(length) stddev_length,
  STDDEV(tonnage) stddev_tonnage
FROM
  [world-fishing-827:scratch_nate.fish_carrier_list_nate_09_2016_plusD]
GROUP BY
  mmsi,
  geartype
order by mmsi
 '''
reefers = Query(q)

Waiting on bqjob_r4c0f0349d68d0f99_00000157d0a9274c_9 ... (0s) Current status: DONE   
Query time: 2.8376660347 seconds.


In [35]:
rows = []
mmsis = []

for c in reefers:
    mmsi = c[0]
    length = c[2]
    tonnage = c[3]
    geartype = "Reefer"
    stddev_length = c[4]
    stddev_tonnage = c[5]
    if length !=None: length=float(c[2])
    if tonnage !=None: tonnage = float(c[3])
    if stddev_length !=None: stddev_length = float(c[4])
    if stddev_tonnage !=None: stddev_tonnage = float(c[5])
    if stddev_length !=None and stddev_length/length > .1:
        length = None # if different entries differ by more than 10 percent, ignore
    if stddev_tonnage !=None and stddev_tonnage/tonnage > 1:
        tonnage = None
    rows.append([geartype,length,tonnage])
    mmsis.append(mmsi)

In [36]:
reefer_frame = pd.DataFrame(rows,index=mmsis,columns=['reefer_geartype','reefer_length', 'reefer_tonnage'])

## Get Countries from 2015 list
Note that if the vessel wasn't scored for 2015, it has a null value in country in this table

In [37]:
q = '''SELECT
  mmsi,
  iso3
FROM
  [world-fishing-827:scratch_david_mmsi_lists.2015_all_fishing_v4]
WHERE
  label IS NOT NULL
  group by mmsi, iso3'''
countries = Query(q)

Waiting on bqjob_r4364775d7d73e431_00000157d0a93276_10 ... (0s) Current status: DONE   
Query time: 2.63207697868 seconds.


In [38]:
mmsis = []
cs = []
for c in countries:
    mmsis.append(c[0])
    cs.append([c[1]])
    
countries_frame = pd.DataFrame(cs,index=mmsis,columns=['country'])

In [39]:
result = eu_frame.join(itu_frame, how='outer')
result = result.join(clav_frame, how='outer')
result = result.join(ccamlr_frame, how='outer')
result = result.join(countries_frame, how='outer')
result = result.join(reefer_frame, how='outer')
result = result.join(squid_frame, how='outer')

# result = pd.concat([result,clav_frame], axis=1, join_axes=[result.index])
# result = result.groupby(result.index).first()
# result = result.drop_duplicates()

In [40]:
result.head()

,eu_geartype,eu_geartype2,eu_length,eu_tonnage,itu_geartype,itu_tonnage,clav_geartype,clav_length,clav_tonnage,ccamlr_geartype,ccamlr_length,ccamlr_tonnage,country,reefer_geartype,reefer_length,reefer_tonnage,squid_geartype
100000005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
100000018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
100000046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
100000668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
100001001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN


In [41]:
rows = []

for i in result.index:
    try:
        mmsi = i
        eu_geartype = result['eu_geartype'].ix[i]
        itu_geartype = result['itu_geartype'].ix[i]
        clav_geartype = result['clav_geartype'].ix[i]
        ccamlr_geartype = result['ccamlr_geartype'].ix[i]
        reefer_geartype = result['reefer_geartype'].ix[i]
        squid_geartype = result['squid_geartype'].ix[i]
        s =  set([eu_geartype, itu_geartype, clav_geartype,ccamlr_geartype,reefer_geartype,squid_geartype])
        if np.nan in s:
            s.remove(np.nan)
        if "Fishing vessel" in s:
            s.remove("Fishing vessel")
        if len(s)!=1:
            geartype = "Fishing vessel"
        else:
            geartype = list(s)[0]

            # this is a way to figure what what the second gearytpe is
        if 'ongline' in str(eu_geartype) and geartype != "Fishing vessel":
            geartype2 = result['eu_geartype2'].ix[i]
        else: 
            geartype2 = geartype

        eu_tonnage = result['eu_tonnage'].ix[i]
        clav_tonnage = result['clav_tonnage'].ix[i]
        itu_tonnage = result['itu_tonnage'].ix[i]
        ccamlr_tonnage = result['ccamlr_tonnage'].ix[i]
        reefer_tonnage = result['reefer_tonnage'].ix[i]
        tonns = [eu_tonnage,clav_tonnage,itu_tonnage,ccamlr_tonnage,reefer_tonnage]
        tonns = np.array([t for t in tonns if t is not None])
        avg = np.nanmean(tonns)
        stddev = np.nanstd(tonns)
        if stddev/avg > .1:
            tonnage = np.nan
        else:
            tonnage = avg

        if np.isnan(tonnage): tonnage = None

        eu_length= result['eu_length'].ix[i]
        clav_length = result['clav_length'].ix[i]
        ccamlr_length = result['ccamlr_length'].ix[i]
        reefer_length = result['reefer_length'].ix[i]
        length = np.array([eu_length,clav_length,ccamlr_length,reefer_length])
        avg = np.nanmean(length)
        stddev = np.nanstd(length)
        if stddev/avg > .1:
            length = np.nan
        else:
            length = avg
        if np.isnan(length): length = None

        country =  result['country'].ix[i]
        if str(country) == 'nan': country = None

        r = [int(mmsi),geartype,geartype2,length,tonnage,country]    
        rows.append(r)
    except:
        print i 
        break


/Users/David/Desktop/Jobs/GlobalFishingWatch/github/vessel-maps/utilities/pipa_paper/venv/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)
/Users/David/Desktop/Jobs/GlobalFishingWatch/github/vessel-maps/utilities/pipa_paper/venv/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:1147: RuntimeWarning: Degrees of freedom <= 0 for slice.
  warnings.warn("Degrees of freedom <= 0 for slice.", RuntimeWarning)


In [42]:
result.ix['224025440']


eu_geartype               NaN
eu_geartype2              NaN
eu_length                 NaN
eu_tonnage                NaN
itu_geartype              NaN
itu_tonnage               NaN
clav_geartype      Longliners
clav_length                25
clav_tonnage               85
ccamlr_geartype           NaN
ccamlr_length             NaN
ccamlr_tonnage            NaN
country                   ESP
reefer_geartype           NaN
reefer_length             NaN
reefer_tonnage            NaN
squid_geartype            NaN
Name: 224025440, dtype: object

In [43]:
header = ['mmsi','geartype','secondary geartype','length','tonnage','country']
with open("fishing_training.csv", 'wb') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    writer.writerows(rows)

In [44]:
ft = pd.read_csv('fishing_training.csv')

In [45]:
ft.groupby(['geartype'])['mmsi'].count()

geartype
Fishing vessel    39197
Longliners         1582
Pole and Line       136
Pots and Traps      398
Purse seines        709
Reefer              694
Set gillnets        576
Squid               164
Trawlers           4948
Trollers             56
Name: mmsi, dtype: int64

In [46]:
ft.groupby(['secondary geartype'])['mmsi'].count()

secondary geartype
Drifting longlines      203
Fishing vessel        39197
Longliners             1154
Pole and Line           136
Pots and Traps          398
Purse seines            709
Reefer                  694
Set gillnets            576
Set longlines           225
Squid                   164
Trawlers               4948
Trollers                 56
Name: mmsi, dtype: int64